In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE, VarianceThreshold, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE, RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline

In [ ]:
#ignore warningsa
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_data():
    df = pd.read_csv('data/new_map.csv', header = 0)
    df.set_index(df.columns[0], inplace=True)
    df.rename_axis("ID", inplace=True)
    
    meta = pd.read_csv('data/samples_wo_UT.csv', header = 0)

    print(df.shape)
    print(meta.shape)
    
    mdf = pd.merge(df, meta, on='ID')
    
    print(mdf.shape)
    
    return mdf

In [ ]:
def preprocess(df, *cols_to_exclude):
    df_copy = df.copy()
    cols_to_exclude = list(cols_to_exclude)
    cols_to_scale = [col for col in df.columns if col not in cols_to_exclude]
    scaler = MinMaxScaler()
    df_copy[cols_to_scale] = scaler.fit_transform(df_copy[cols_to_scale])       
    return df_copy

In [ ]:
df = load_data()
df = df.drop(['ID'],axis = 1)
df = df.drop(['path', 'group', 'name','phenotype', 'genotype', 'cell', 'status',],axis=1)
x_train = preprocess(df,'target0')
y_train = df['target0']
x_train = x_train.drop('target0',axis = 1)
print(x_train.shape)
print(y_train.shape)
X = x_train
y = y_train

In [ ]:
# feature selection method
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)
from xgboost import XGBClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.stats import spearmanr
from itertools import combinations

# Try with random features
# Example: Replace this with actual data loading
# X = <your_features>
# y = <your_labels>
#X = np.random.rand(32, 40000)  # Placeholder dataset with 32 samples and 40000 features
#y = np.random.randint(0, 2, 32)  # Placeholder binary labels

df = load_data()
df = df.drop(['ID'],axis = 1)
df = df.drop(['path', 'group', 'name','phenotype', 'genotype', 'cell', 'status',],axis=1)
x_train = preprocess(df,'target0')
y_train = df['target0']
x_train = df.drop('target0',axis = 1)
print(f'shape 1 {x_train.shape}')
print(y_train.shape)
X = x_train
y = y_train

# Check for class imbalance
print(f"Original class distribution: {Counter(y)}")

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "AdaBoost": AdaBoostClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True),
    "Naive Bayes": GaussianNB(),
}

# Scaling features
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

#X = pd.DataFrame(X)
# Stratified K-Fold Cross Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Feature Selection and Model Evaluation
results = {model_name: [] for model_name in models.keys()}
final_feature_names = []
common_features = None
feature_rankings = []
selected_features_list = []

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Apply SMOTE for oversampling the minority class
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    print(f"Class distribution after SMOTE: {Counter(y_train)}")

    # Stage 1: Variance Threshold
    selector = VarianceThreshold(threshold=0.01)
    X_train = selector.fit_transform(X_train)
    X_test = selector.transform(X_test)
    feature_names = X.columns[selector.get_support()]
    print(f"Stage 1 - Variance Threshold: {len(feature_names)} features selected")
    print(X_train.shape)
    print(X_test.shape)
    
    # Stage 2: Univariate Feature Selection (ANOVA F-test)
    selector = SelectKBest(score_func=f_classif, k=5000)  # Select top 2000 features
    X_train = selector.fit_transform(X_train, y_train)
    X_test = selector.transform(X_test)
    feature_names = feature_names[selector.get_support()]
    print(f"Stage 2 - SelectKBest: {len(feature_names)} features selected")
    print(X_train.shape)
    print(X_test.shape)

    # Stage 3: Recursive Feature Elimination with Random Forest
    rfc = RandomForestClassifier(n_estimators=1000, random_state=42)
    rfc.fit(X_train, y_train)
    importances = rfc.feature_importances_
    indices = np.argsort(importances)[::-1][:1000]  # Select top 500 features
    X_train = X_train[:, indices]
    X_test = X_test[:, indices]
    feature_names = feature_names[indices]
    print(f"Stage 3 - Recursive Feature Elimination: {len(feature_names)} features selected")
    print(X_train.shape)
    print(X_test.shape)

    # Stage 4: Final Feature Reduction (Top 100 features using Lasso)
    lasso = Lasso(alpha=0.05, max_iter=1000)
    lasso.fit(X_train, y_train)
    lasso_mask = np.argsort(np.abs(lasso.coef_))[::-1][:500]  # Indices of top 100 features
    X_train = X_train[:, lasso_mask]
    X_test = X_test[:, lasso_mask]
    final_feature_names = feature_names[lasso_mask]

    feature_rankings.append(final_feature_names)
    selected_features_list.append(set(final_feature_names))

    #print(f"Stage 4 - Final Feature Set: {len(final_feature_names)} features selected")
    print(X_train.shape)
    print(X_test.shape)
    #final_feature_names = feature_names
    #print(f'Final features list {final_feature_names.tolist()}')
  #  # Stage 4: Final Feature Reduction (Top 100 features)
  #  X_train = X_train[:, :100]
  #  X_test = X_test[:, :100]
  #  final_feature_names = feature_names[:100]
  #  print(f"Stage 4 - Final Feature Set: {len(final_feature_names)} features selected")
    # Track features across folds
    if common_features is None:
        common_features = set(final_feature_names)
    else:
        common_features.intersection_update(final_feature_names)
  
    # Bootstrapping for Model Evaluation
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        # Performance Metrics
        accuracy = accuracy_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)
        specificity = recall_score(y_test, y_pred, pos_label=0)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        results[model_name].append({
            "accuracy": accuracy,
            "sensitivity": sensitivity,
            "specificity": specificity,
            "f1_score": f1,
            "auc": roc_auc,
        })

# Compute Spearman Rank Stability
all_features = list(set().union(*feature_rankings))
rank_matrices = np.zeros((len(feature_rankings), len(all_features)))
for i, ranking in enumerate(feature_rankings):
    for j, feature in enumerate(all_features):
        if feature in ranking:
            rank_matrices[i, j] = ranking.tolist().index(feature) + 1
        else:
            rank_matrices[i, j] = len(ranking) + 1  # Assign lowest rank if not present

spearman_correlations = []
for i in range(len(rank_matrices) - 1):
    corr, _ = spearmanr(rank_matrices[i], rank_matrices[i + 1])
    spearman_correlations.append(corr)

mean_spearman_stability = np.mean(spearman_correlations)
print(f"Spearman Rank Stability: {mean_spearman_stability:.4f}")

# Summary of Results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric_name in ["accuracy", "sensitivity", "specificity", "f1_score", "auc"]:
        mean_metric = np.mean([m[metric_name] for m in metrics])
        print(f"  {metric_name}: {mean_metric:.4f}")

# Display final selected feature names
print("Final Selected Features (Top 100):", final_feature_names.tolist())
# Output common features across all folds
common_features = list(common_features)
print("\n--- Common Features Across All Folds ---")
print(common_features)


In [ ]:
#model training and validation with common_features set

X_features = X[common_features]
#random_features = np.random.choice(X.columns, size=58, replace=False)
#X_features = X[random_features]

print(X_features.shape)
print(y.shape)

# Check for class imbalance
print(f"Original class distribution: {Counter(y)}")

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "AdaBoost": AdaBoostClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True),
    "Naive Bayes": GaussianNB(),
}

# Scaling features
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

#X = pd.DataFrame(X)
# Stratified K-Fold Cross Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Feature Selection and Model Evaluation
results = {model_name: [] for model_name in models.keys()}

for train_idx, test_idx in skf.split(X_features, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Apply SMOTE for oversampling the minority class
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    print(f"Class distribution after SMOTE: {Counter(y_train)}")

    # Bootstrapping for Model Evaluation
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        # Performance Metrics
        accuracy = accuracy_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)
        specificity = recall_score(y_test, y_pred, pos_label=0)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        results[model_name].append({
            "accuracy": accuracy,
            "sensitivity": sensitivity,
            "specificity": specificity,
            "f1_score": f1,
            "auc": roc_auc,
        })


# Summary of Results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric_name in ["accuracy", "sensitivity", "specificity", "f1_score", "auc"]:
        mean_metric = np.mean([m[metric_name] for m in metrics])
        print(f"  {metric_name}: {mean_metric:.4f}")


In [ ]:
#SHAP with Naive Bayes
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_selected = X[common_features]
print(X_selected.shape)
print(y.shape)

# Train-test split
#X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Initialize Naïve Bayes model
model = GaussianNB()

# Train model
model.fit(X_selected, y)

# Compute SHAP values
explainer = shap.KernelExplainer(model.predict, X_selected)
shap_values = explainer.shap_values(X_selected)

# SHAP Summary Plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_selected, show=False)
plt.title("SHAP Summary Plot - Naïve Bayes")
plt.show()

shap.plots.violin(shap_values,X_selected, show=False)
shap.summary_plot(shap_values,X_selected, show=False, plot_type="bar")


In [ ]:
shap.summary_plot(shap_values,X_selected, show=False, plot_type="bar")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load data
df = pd.read_csv("data/plot.csv")  # Ensure correct formatting
print("Original Data Shape:", df.shape)

# Transpose data to have mRNAs as rows and Usher/Control as columns
df_t = df.T  # Transpose the dataframe
df_t.columns = df_t.iloc[0]  # Set mRNA names as column headers
df_t = df_t.iloc[1:]  # Remove the first row (old column headers)
df_t = df_t.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric

#print("Transposed Data:\n", df_t)
#print("Transposed Data Shape:", df_t.shape)  # Should be (num_samples, 58)

# Apply Log Transformation for better visualization
df_t_log = np.log1p(df_t)  # log1p to prevent log(0) errors
#print("Log-Transformed Data:\n", df_t_log)

# Create Heatmap
plt.figure(figsize=(14, 8))  # Adjusted figure size for better visualization
ax = sns.heatmap(
    df_t_log, 
    cmap="OrRd", 
    annot=False, 
    linewidths=0.5, 
    yticklabels=True  # Ensure y-axis labels are shown
)

# Formatting
#plt.title("Heatmap of Log-Scaled mRNA Expression in Usher vs. Control", fontsize=16)
plt.xlabel("mRNAs", fontsize=12)
plt.ylabel("Expression Level", fontsize=12)
plt.xticks(rotation=90)  # Rotate sample labels for better readability

# Ensure all 58 mRNA labels appear on the y-axis
ax.set_yticks(np.arange(df_t_log.shape[0]) + 0.5)  # Center y-ticks
ax.set_yticklabels(df_t_log.index, fontsize=10, rotation=0)  # Show all mRNA names

plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.savefig("heatmap_light_palette.jpg", dpi=600,format="jpg", bbox_inches="tight")
plt.show()
